# GAIA-DRL: DDPG Training
This notebook demonstrates how to train the GAIA-DRL agent using synthetic IoT and geospatial data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from gym import spaces

# Carregar amostras de vetor de estado com NDVI (Vt)
df = pd.read_csv('../data/Vt_samples.csv')
df.head()

In [ ]:
class GAIAEnv(gym.Env):
    def __init__(self, df):
        super(GAIAEnv, self).__init__()
        self.df = df
        self.max_steps = len(df)
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)
        self.observation_space = spaces.Box(low=0.0, high=1.0, shape=(5,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.current_step = 0
        return self._get_obs()

    def _get_obs(self):
        row = self.df.iloc[self.current_step]
        return np.array([row['Rt'], row['Et'], row['Lt'], row['It'], row['Vt']], dtype=np.float32)

    def step(self, action):
        row = self.df.iloc[self.current_step]
        reward = 0.25 * row['Et'] + 0.25 * row['Rt'] - 0.2 * row['Lt'] - 0.2 * row['It'] + 0.1 * row['Vt']
        self.current_step += 1
        done = self.current_step >= self.max_steps
        return self._get_obs(), reward, done, {}

In [ ]:
env = GAIAEnv(df)
obs = env.reset()
print('Initial Observation:', obs)

for _ in range(5):
    action = env.action_space.sample()
    obs, reward, done, _ = env.step(action)
    print(f'Action: {action}, Reward: {reward}')